In [31]:
import selenium
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from openpyxl import Workbook

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pyautogui
import mouseinfo

import time
import datetime
import requests
import random


import os
import pandas as pd
from tqdm import tqdm

In [34]:
url = "https://www.diningcode.com/list.dc?query="
places = ["화양동", "군자동", "광장동"]

In [110]:
options = Options()
# Chrome의 binary path 지정
options.binary_location = (
    "/Users/powerjsv/Desktop/Google Chrome.app/Contents/MacOS/Google Chrome"
)

# 불필요한 에러 메시지 삭제
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# 최대화
options.add_argument('--start-maximized')

chrome_driver_path = (
    "/Users/powerjsv/Desktop/project/capstone/chromedriver-mac-arm64/chromedriver"
)

driver = webdriver.Chrome(chrome_driver_path, chrome_options=options)
driver.implicitly_wait(2)

/var/folders/9q/1dly5q8d5j9dmt8t1fwf05r80000gn/T/ipykernel_38881/2833979925.py:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_path, chrome_options=options)


In [99]:
def remove_fucking_ad():
    ad_path = ["#dismiss-button"]

    try:
        driver.find_element(By.CSS_SELECTOR, ad_path).click()
        print("찾았다 씨발 ㅋㅋ")
        time.sleep(3)
    except:
        print("cannot find ad")
        return

In [111]:
for place in places:
    # driver.get(url+place)
    driver.get(url + "화양동")
    remove_fucking_ad()
    # # 검색 결과 더 보기
    # xpath_view_more_search = '//*[@id="root"]/div/main/div[2]/div[3]/div/a'
    # driver.find_element_by_xpath(xpath_view_more_search).click()

    # 20개씩 5번 검색결과 더 보기 클릭
    xpath_view_20_more = '//*[@id="map"]/button[2]/span'
    for _ in range(4):
        driver.find_element_by_xpath(xpath_view_20_more).click()
        time.sleep(random.uniform(1,3))
    # 각각의 식당 클릭
    cnt=0
    restaurant_find_selector = "div.RHeader > div > div.InfoHeader"

    for i in driver.find_elements(By.CSS_SELECTOR, restaurant_find_selector):
        # remove_fucking_ad()
        cnt+=1
        i.click()
        time.sleep(random.uniform(1,3))
        tabs = driver.window_handles

        # 클릭시 리뷰가 새 창으로 뜨기 때문에 창을 이동해줌
        driver.switch_to.window(tabs[1])
        print("switch!")

        # 리뷰 더보기 버튼 안 나올때까지 클릭
        while True:
            try:
                more_review_selector = "#div_more_review > button"
                driver.find_element(By.CSS_SELECTOR,more_review_selector).click()
                time.sleep(random.uniform(1,3))
            except:
                break
        restaurant=[]
        review=[]

        # 창 닫고 원래 창으로 돌아옴
        driver.close()
        
        if tabs[1] :
            print("찾앗다 개 씨발")
            driver.close()
        driver.switch_to.window(tabs[0])
        if cnt == 3:
            break
    break

cannot find ad
switch!
찾앗다 개 씨발


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.58)


In [105]:
remove_fucking_ad()

cannot find ad


In [112]:
tabs=driver.window_handles
print(tabs)

['72C0F07297D92785562971DAA3CBFAB1']


In [101]:
driver.switch_to_window(tabs[0])

/var/folders/9q/1dly5q8d5j9dmt8t1fwf05r80000gn/T/ipykernel_38881/2204257098.py:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])


In [103]:
driver.switch_to_window(tabs[1])

/var/folders/9q/1dly5q8d5j9dmt8t1fwf05r80000gn/T/ipykernel_38881/1828234608.py:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1])


In [ ]:
reviewer_name_selector = ".latter-graph"

In [27]:
review_selector = "#div_review"
tmp=driver.find_elements(By.CSS_SELECTOR,".latter-graph")
for i in range(10):
    print(tmp[5].text)

이구아날
평균 별점 4.1   평가 577   팔로워 2
4점 맛 맛있음 가격 보통 응대 친절함
바나나 푸딩 맛집이라고 들어서 갔는데 맛은 생각보다 보통이었음
바나나푸딩 아메리카노
차모임 혼카페 데이트 간식 조용한 깔끔한 경관/야경이좋은 지역주민이찾는
2023년 10월 3일
이구아날
평균 별점 4.1   평가 577   팔로워 2
4점 맛 맛있음 가격 보통 응대 친절함
바나나 푸딩 맛집이라고 들어서 갔는데 맛은 생각보다 보통이었음
바나나푸딩 아메리카노
차모임 혼카페 데이트 간식 조용한 깔끔한 경관/야경이좋은 지역주민이찾는
2023년 10월 3일
이구아날
평균 별점 4.1   평가 577   팔로워 2
4점 맛 맛있음 가격 보통 응대 친절함
바나나 푸딩 맛집이라고 들어서 갔는데 맛은 생각보다 보통이었음
바나나푸딩 아메리카노
차모임 혼카페 데이트 간식 조용한 깔끔한 경관/야경이좋은 지역주민이찾는
2023년 10월 3일
이구아날
평균 별점 4.1   평가 577   팔로워 2
4점 맛 맛있음 가격 보통 응대 친절함
바나나 푸딩 맛집이라고 들어서 갔는데 맛은 생각보다 보통이었음
바나나푸딩 아메리카노
차모임 혼카페 데이트 간식 조용한 깔끔한 경관/야경이좋은 지역주민이찾는
2023년 10월 3일
이구아날
평균 별점 4.1   평가 577   팔로워 2
4점 맛 맛있음 가격 보통 응대 친절함
바나나 푸딩 맛집이라고 들어서 갔는데 맛은 생각보다 보통이었음
바나나푸딩 아메리카노
차모임 혼카페 데이트 간식 조용한 깔끔한 경관/야경이좋은 지역주민이찾는
2023년 10월 3일
이구아날
평균 별점 4.1   평가 577   팔로워 2
4점 맛 맛있음 가격 보통 응대 친절함
바나나 푸딩 맛집이라고 들어서 갔는데 맛은 생각보다 보통이었음
바나나푸딩 아메리카노
차모임 혼카페 데이트 간식 조용한 깔끔한 경관/야경이좋은 지역주민이찾는
2023년 10월 3일
이구아날
평균 별점 4.1   평가 577   팔로워 2
4점 맛 맛있음 가격 보통 응대 친절함
바나나 푸딩 맛집이라고 들어서 갔는데 맛

In [8]:
a=pd.DataFrame()

In [22]:
c=[]

In [23]:
b={"restaurant": "Burgerking"}

In [24]:
c.append(b)

In [25]:
c

[{'restaurant': 'Burgerking'}]

In [20]:
a=pd.concat(b)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid